### Portfolio Diversification using K-Means

This work presents an approach to using the K-means algorithm for stock classification, with the aim of helping investors to diversify their investment portfolios. It´s divided into 4 parts:

1. Introducing the k-means algorithm
2. Clustering of stocks by return and volatility
3. Clustering of shares by price-earnings ratio and dividend rate
4. 3-dimensional analysis using the k-means++ algorithm by return, volatility, and PER.

In [1]:
#Import the libraries that we are going to need to carry out the analysis:
import numpy as np 
import pandas as pd
import pandas_datareader as dr
import yfinance as yf

from pylab import plot,show
from matplotlib import pyplot as plt
import plotly.express as px

from numpy.random import rand
from scipy.cluster.vq import kmeans,vq
from math import sqrt
from sklearn.cluster import KMeans 
from sklearn import preprocessing

ModuleNotFoundError: No module named 'pandas_datareader'